<a href="https://colab.research.google.com/github/WilliamHackspeare/chess-environment/blob/main/Chess_Environment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chess Environment

This project is meant to showcase a playground for human and programmatic chess play, building upon the python-chess and stockfish modules.



In [1]:
!pip install python-chess stockfish
!wget https://github.com/official-stockfish/Stockfish/releases/latest/download/stockfish-ubuntu-x86-64-avx2.tar
!tar -xvf stockfish-ubuntu-x86-64-avx2.tar

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 kB 5.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for chess: filename=chess-1.11.1-py3-none-any.whl size=148497 sha256=00d04e5716a16a48b3e779899d1fbc0d62d6999afbd14791ab9d91dc67a7a3a2
  Stored in directory: /root/.cache/pip/wheels/2e/2d/23/1bfc95db984ed3ecbf6764167dc7526d0ab521cf9a9852544e
Successfully built chess
--2024-12-21 07:59:10--  https://github.com/official-stockfish/Stockfish/releases/latest/download/stockfish-ubuntu-x86-64-avx2.tar
Resolving github.com (github.com)... 140.82.116.3
Connecting to github.com (github.com)|140.82.116.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/official-stockfish/Stockfish/releases/download/sf_17/stockfish-ubuntu-x86-64-avx2.tar [following]
--2024-12-21 07:59:10--  https://github.com/official-stockfish/Stockfish/releases/download/sf_17/stockfish-ubuntu-x86-64-avx2.tar
Reusing existing connection to github

In [2]:
import chess
import chess.svg
from IPython.display import display, SVG, clear_output
import time
import stockfish

In [3]:
class GamePlay:
    """
    This class manages a chess game between a human player and the Stockfish engine.
    """
    def __init__(self):
        """
        Initializes the chessboard, Stockfish engine, and game state.
        """
        self.stockfish = stockfish.Stockfish(path="/content/stockfish/stockfish-ubuntu-x86-64-avx2")  # Path to the Stockfish engine
        self.board = chess.Board()  # Create a new chessboard
        self.move_list = []  # List to store the moves made in the game
        self.current_turn_white = True  # Start with white's turn

    def string_to_move(self, move_string):
        """
        Converts a move string in UCI format (e.g., 'e2e4') to a chess.Move object.
        """
        return chess.Move.from_uci(move_string)

    def make_move(self, move_string):
        """
        Makes a move on the chessboard.
        """
        move = self.string_to_move(move_string)
        if move in self.board.legal_moves:
            self.board.push(move)  # Make the move
            self.move_list.append(move)  # Add the move to the list
            self.current_turn_white = not self.current_turn_white  # Switch turns
        else:
            raise Exception("Illegal move")  # Raise an exception for illegal moves

    def stockfish_move(self):
        """
        Gets the best move from Stockfish and makes it on the board.
        """
        self.stockfish.set_fen_position(self.board.fen())  # Update Stockfish with the current board position
        move = self.stockfish.get_best_move()  # Get the best move from Stockfish
        self.make_move(move)  # Make the move on the board

    def status(self):
        """
        Checks if the game is over.
        """
        return self.board.is_game_over()

    def result(self):
        """
        Returns the result of the game (e.g., '1-0', '0-1', '1/2-1/2').
        """
        return self.board.outcome()

    def display(self):
        """
        Displays the chessboard in an IPython environment.
        """
        return display(SVG(chess.svg.board(board=self.board, lastmove=self.board.peek(), size=400)))

    def move_cycle(self, move_string):
        """
        Handles a single move cycle: human move, display, Stockfish move, display.
        """
        self.make_move(move_string)  # Make the human's move
        self.display()  # Display the board
        time.sleep(5)  # Wait for 5 seconds
        clear_output(wait=True)  # Clear the output

        if not self.status():  # If the game is not over
            self.stockfish_move()  # Make Stockfish's move
            if self.status():  # If the game is over after Stockfish's move
                print(self.result())  # Print the result
        else:
            print(self.result())  # If the game was over after the human's move, print the result
        self.display()  # Display the board again

    def autoplay(self):
        """
        Lets Stockfish play against itself.
        """
        while not self.status():
            self.stockfish_move()
            self.display()
            clear_output(wait=False)  # Clear the output without waiting
        self.display()
        return self.result()  # Return the result of the game